# Segmenting and clustering neighborhood in Toronto
With the help of Notebook, the code is build to scrape the mentioned Wikipedia page in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe as mentioned.
## Importing and Installing the required libraries 

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install folium
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 93 kB 2.7 MB/s eta 0:00:011
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                                          /failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.9|3.6.9|3.6.9|3.6.9|>=3.6,<3.7.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|>=3.7,<3.8.0a0|>=2.7,<2.8.0a0|>=3.5,<3.6.0a0|3.4.*',build='1_73_pypy|2_73_pypy|3_73_pypy|0_73_pypy']
  - rsa -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']

Your python: python=3.7

If python is on the left-most side of the chain,


Package grpcio conflicts for:
tensorflow-estimator -> grpcio[version='>=1.8.6']
ibm-wsrt-py37main-main -> grpcio==1.27.2[build=*]
tensorflow-base -> grpcio[version='>=1.14.1,<2.0a0|>=1.14.1|>=1.16.1|>=1.8.6']
tensorboard -> grpcio[version='>=1.14|>=1.24.3|>=1.6.3']

Package async-timeout conflicts for:
aiohttp -> async-timeout[version='<4.0,>=3.0|>=3.0,<4.0|>=1.2,<3.0|>=1.2.0']
ibm-wsrt-py37main-main -> async-timeout==3.0.1[build=*]

Package matplotlib conflicts for:
nltk -> matplotlib
scikit-image -> matplotlib[version='>=1.1|>=1.3.1|>=2.0.0']
arcgis=1.6.0 -> matplotlib
bokeh -> matplotlib
ibm-wsrt-py37main-main -> matplotlib==3.2.2[build=*]
seaborn -> matplotlib[version='>=1.4.3|>=2.1.2']

Package chardet conflicts for:
aiohttp -> chardet[version='>=2.0,<4.0']
requests -> chardet[version='>=3.0.2,<3.1.0|>=3.0.2,<4']
ibm-wsrt-py37main-main -> chardet==3.0.4[build=*]

Package attrs conflicts for:
pytest -> attrs[version='>=17.2.0|>=17.4.0|>=17.4.0,<=19.1.0']
ibm-wsrt-py37main-main -> 

Folium installed
Libraries imported.


## Scraping the wikipedia page inorder to get the table of postal codes of Canada

Using BeautifulSoup Library of Python, web scraping of table is done from the Wikipedia. The title of the webpage is printed to check whether the page has craped successfully and then the postal code of canada is printed.

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
#print(soup.title)
#from IPython.display import display_html
tab = str(soup.table)
#display_html(tab,raw=True)
dfs = pd.read_html(tab)
df=dfs[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


The HTML page is converted to dataframe for cleaning and preprocessing


In [6]:
# Dropping the rows where Borough is 'Not assigned'
df1 = df[df.Borough != 'Not assigned']

# Combining the neighbourhoods with same Postalcode
df2 = df1.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)

# Replacing the name of the neighbourhoods which are 'Not assigned' with names of Borough
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned',df2['Borough'], df2['Neighbourhood'])

df2

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


To display shape of Dataframe

In [7]:
df2.shape

(103, 3)